In [1]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 88.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.

In [2]:
import pandas as pd
import tensorflow.keras.backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV

In [3]:
ds = pd.read_csv('/kaggle/input/ds-card12/autos.csv', encoding = 'ISO-8859-1')

In [4]:
toDrop = ['dateCrawled', 'name', 'seller', 'offerType', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen']
for i in toDrop:
    ds = ds.drop(i, axis=1)

In [5]:
ds = ds.loc[ds['price'] > 10]
ds = ds.loc[ds['price'] < 350000]

In [6]:
values = {
    'vehicleType': ds['vehicleType'].mode()[0],
    'gearbox': ds['gearbox'].mode()[0],
    'model': ds['model'].mode()[0],
    'fuelType': ds['fuelType'].mode()[0],
    'notRepairedDamage': ds['notRepairedDamage'].mode()[0]
}
ds = ds.fillna(value = values)

In [7]:
X = ds.iloc[:, 1:12].values 
y = ds.iloc[:, 0].values

In [8]:
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough')

In [9]:
X = onehotencoder.fit_transform(X).toarray() #aplica a transformação à X

In [10]:
def create_network(loss):
    k.clear_session()
    network = Sequential()
    network.add(InputLayer(shape = (316,)))
    network.add(Dense(units = 158, activation='relu'))
    network.add(Dense(units = 158, activation='relu'))
    network.add(Dense(units = 1, activation='linear'))

    network.compile(optimizer='adam', loss=loss, metrics=['mean_absolute_error'])
    return network

In [11]:
network = KerasRegressor(model = create_network, batch_size=300, epochs = 100)

In [12]:
params = {'model__loss': ['mean_squared_error', 'mean_absolute_error',
                       'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
                       'squared_hinge']}

In [13]:
grid_search = GridSearchCV(estimator = network, param_grid=params, 
                           scoring='neg_mean_absolute_error',cv = 5)

In [14]:
grid_search = grid_search.fit(X, y)

Epoch 1/100


I0000 00:00:1724858640.670296      70 service.cc:145] XLA service 0x7ce76c0048d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1724858640.670355      70 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


100/959 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 104181416.0000 - mean_absolute_error: 6065.8501

I0000 00:00:1724858641.594040      70 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 75078184.0000 - mean_absolute_error: 4727.7368
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 51726232.0000 - mean_absolute_error: 3790.7861
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 45415052.0000 - mean_absolute_error: 3421.7881
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 42308768.0000 - mean_absolute_error: 3217.6187
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 40128776.0000 - mean_absolute_error: 3072.9985
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 40984292.0000 - mean_absolute_error: 3032.1316
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 37918972.0000 - mean_absolute_error: 2899.2639
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 39950144.0000 - mean_absolute_error: 2909.3718
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 36387452.0000 - mean_absolute_error: 2868.9019
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━

In [15]:
best_params = grid_search.best_params_
best_params

{'model__loss': 'mean_absolute_error'}

In [16]:
best_score = grid_search.best_score_
best_score

-2343.410417147609